# Limpieza de Datos

En este notebook vamos a crear 2 funcciones de limpieza de datos, una para el dataframe de ventas y otra para el dataframe de productos. El proceso consiste en: identificar y corregir errores(errores de entrada, como errores tipográficos o de formato), inconsistencias y redundancias en conjuntos de datos, manejar los valores faltantes, manejar los duplicados, eliminar valores que no aportan informacion, cambiar los tipos de variables, crear nuevas columnas basadas en las existentes o para asignar categorías a los datos existentes. Un conjunto de datos limpio y preciso mejora la confiabilidad de los resultados y contribuye a la toma de decisiones informada.

IMPORTAMOS LAS LIBRERIAS QUE NECESITAMOS

In [1]:
import pandas as pd
import numpy as np

LEEMOS LOS CSV 

In [2]:
data = pd.read_csv('data_ventas.csv')
productos = pd.read_csv('Productosf.csv')

C:\Users\heuni\AppData\Local\Temp\ipykernel_7008\2973436202.py:1: DtypeWarning: Columns (7,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_ventas.csv')


## Limpieza de DataFrame Ventas

In [5]:
def limpieza(data):
    #RELLENAMOS NAN
    data['CANAL_VENTA']=data['CANAL_VENTA'].fillna('')
    data['NOMBRE_PRODUCTO']=data['NOMBRE_PRODUCTO'].fillna('')
    data['CADUCIDAD_2']=data['CADUCIDAD_2'].fillna(' ')
    data['FECHA_COMPRA']=data['FECHA_COMPRA'].fillna(' ')
    data['ID_PRODUCTO'] = data.groupby('NOMBRE_PRODUCTO')['ID_PRODUCTO'].fillna(method='ffill')
    
    #QUITAMOS LOS DATOS QUE NO NECESITAMOS(las filas donde tenemos: bolsas, palets, descuento, insumo de cafeteria o filas con valores faltantes, las columnas que no nos aportan ninguna informacion valiosa)
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('Bolsa de plastico')]
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('Palets')]
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('DESCUENTO')]
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('AJUSTE A TICKETS')]
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('Insumo de cafeteria')]
    data = data[data['ID_PRODUCTO'] != '2']
    data = data[data['ID_PRODUCTO'] != '3']
    data.drop(data.tail(1).index, inplace=True)
    data.drop(['A', 'CADUCIDAD', 'D', 'E'], axis = 1 , inplace=True)
    
    #TRANSFORMAMOS LAS COLUMNAS FECHA_COMPRA Y CADUCIDAD EN FORMATO DATETIME
    data['FECHA_COMPRA'] = pd.to_datetime(data['FECHA_COMPRA'], format = ' %d/%m/%Y', errors='coerce')
    data['CADUCIDAD_2'] = pd.to_datetime(data['CADUCIDAD_2'], format = '%m/%d/%Y', errors='coerce')
    
    #LLENAMOS LAS FILAS DONDE LA FECHA DE CADUCIDAD ES EL AÑO 1970 CON LA ULTIMA FECHA DE CADUCIDAD DE ESTE PRODUCTO
    productos_con_error = data[data['CADUCIDAD_2'].dt.year == 1970]
    if not productos_con_error.empty:
        ultima_fecha_por_producto = data.groupby('ID_PRODUCTO')['CADUCIDAD_2'].max().reset_index()

        for index, row in productos_con_error.iterrows():
            producto_id = row['ID_PRODUCTO']
            nuevas_fechas_caducidad = ultima_fecha_por_producto.loc[ultima_fecha_por_producto['ID_PRODUCTO'] == producto_id, 'CADUCIDAD_2']
        
            if not nuevas_fechas_caducidad.empty:
                nueva_fecha_caducidad = nuevas_fechas_caducidad.values[0]
                data.at[index, 'CADUCIDAD_2'] = nueva_fecha_caducidad
    
    #CREAMOS INDICES PARA LOS CLIENTES 
    data['INDEX_CLIENTES'] = (data['ID_PRODUCTO'].str.contains('Código:') | data['ID_PRODUCTO'].str.startswith('Código:')).cumsum()
    data.drop(data[data['ID_PRODUCTO'] == 'Código:'].index, inplace=True)
    
    #CREAMOS ETIQUETAS POR CANAL DE VENTA (TENEMOS 3 CANALES)
    def asignar_index(canal):
        if canal.startswith('TB'):
            return 1 #Ventas tienda"
        elif canal.startswith('E'):
            return 2 #Ventas Online"
        elif canal.startswith(('SR', 'S9', 'VA', 'C2','R')) or canal.isdigit():
            return 3 #Ventas Almacen"
        else:
            return 0  # Otro caso   
    data['ETIQUETA_CANAL'] = data['CANAL_VENTA'].apply(asignar_index)
    
    #CAMBIAMOS COLUMNAS CATEGORICAS A NUMERICAS
    columnas_numericas = ['CANTIDAD', 'PRECIO_UNIDAD', 'PRECIO_TOTAL']
    data[columnas_numericas] = data[columnas_numericas].apply(pd.to_numeric, errors= 'coerce')
    data.reset_index(drop=True, inplace=True)
    
    #LIMPIAMOS Y ADAPTAMOS OTROS DATOS FALTANTES QUE HEMOS IDENTIFICADO DURANTE EL TRABAJO
    data = data[data['ID_PRODUCTO'] != '00000000']
    data['PRECIO_TOTAL'].fillna(data['CANTIDAD'] * data['PRECIO_UNIDAD'], inplace=True)
    data['CANTIDAD'].fillna(data['PRECIO_TOTAL'] / data['PRECIO_UNIDAD'], inplace=True)
    data.dropna(subset=['CANTIDAD'], inplace=True)

    return data

## Limpieza de DataFrame de Productos

In [6]:
def limpieza2(data):
    #CAMBIAMOS LOS NOMBRES DE LAS COLUMNAS
    data = data.rename(columns={"Artículo": "ID_PRODUCTO", "Concepto/Descripción": "NOMBRE_PRODUCTO", "Familia": "CATEGORIA", "Subfamilia": "PROVEEDOR", "Unid. Vendidas": "UNIDADES_VENDIDAS"})
    
    #RELLENAMOS LOS NAN
    data['NOMBRE_PRODUCTO']=data['NOMBRE_PRODUCTO'].fillna('')
    
    #QUITAMOS LOS DATOS QUE NO NECESITAMOS
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('Bolsa de plastico')]
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('Palets')]
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('DESCUENTO')]
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('Bolsas Termicas')]
    data = data[~ data['NOMBRE_PRODUCTO'].str.contains('Insumo de cafeteria')]
    data = data[data['ID_PRODUCTO'] != '00000000']
    data = data[data['NOMBRE_PRODUCTO'] != 'a']
    data.drop(data.tail(1).index, inplace=True)
    data.drop(['Total Euros IVA'], axis = 1 , inplace=True)
    data.reset_index(drop=True, inplace=True)
    
    #ORGANIZAMOS LAS CATEGORIAS PORQUE ALGUNAS SE REPITEN
    mapeo_categorias = {
    'Pepinos': 'Frutas y Verduras',
    'Tomates': 'Frutas y Verduras',
    'Setas': 'Frutas y Verduras',
    'Cerveza': 'Bebidas alcoholicas',
    'VINO': 'Bebidas alcoholicas',
    'Vodka': 'Bebidas alcoholicas',
    'Brandy': 'Bebidas alcoholicas',
    'salsa': 'Salsa',
    'Barquillos' : 'Dulces',
    'Bizcochos' : 'Dulces',  
    'Galletas' : 'Dulces',
    'Bombones' : 'Dulces', 
    'Tartas' : 'Dulces', 
    'Te,Café' : 'Te,Cafe',
    'Zumo' : 'Agua y refrescos', 
    'Productos Lacteos Congelados' : 'Productos lacteos', 
    'Salchichon RO' : 'Salchichon i embutidos', 
      }
    data['CATEGORIA'] = data['CATEGORIA'].replace(mapeo_categorias)
    data['ID_PRODUCTO'] =data['ID_PRODUCTO'].fillna('00010901')
    return data


## DATAFRAMES LIMPIOS

In [ ]:
ventas = limpieza(data)
info_productos = limpieza2(productos)

## DATOS UNIDOS

In [ ]:
# VAMOS A UNIR LOS DOS DATAFRAMES EN SOLO UNO POR LA COLUMNA COMUN ID_PRODUCTO
tienda = pd.merge(ventas, info_productos, on='ID_PRODUCTO', how='inner')
tienda.drop(['NOMBRE_PRODUCTO_y', 'PROVEEDOR', 'UNIDADES_VENDIDAS', 'PRECIO_UNIDAD', 'CANAL_VENTA',], axis=1, inplace=True)
tienda.rename(columns={'NOMBRE_PRODUCTO_x': 'NOMBRE_PRODUCTO', 'CADUCIDAD_2': 'CADUCIDAD'}, inplace=True)
tienda.drop('NOMBRE_PRODUCTO', axis=1, inplace=True)

In [19]:
tienda.to_csv('tienda.csv', index=False)

# CONCLUSIÓN

La información proporcionada por la empresa ha sido extremadamente valiosa, ya que nos brindó la oportunidad de trabajar con datos reales. Sin embargo, nos enfrentamos a dataframes bastante desordenados, lo que requirió un extenso proceso de limpieza. A lo largo de nuestro trabajo, hemos tenido que regresar varias veces para incorporar nuevos elementos de limpieza a medida que descubríamos más desafíos en los datos.
Durante este proceso, hemos llegado a comprender la importancia crítica de la limpieza de datos. Su impacto directo en la calidad, confiabilidad y utilidad de la información no puede subestimarse. Los conjuntos de datos limpios no solo facilitan una manipulación más eficiente, sino que también posibilitan una extracción de información más rápida y precisa.
La observación de que trabajar con datos limpios reduce significativamente la necesidad de corregir errores o abordar problemas de calidad durante las etapas posteriores del análisis es crucial. Esto no solo mejora la eficiencia operativa, sino que también asegura la precisión de los resultados analíticos, fundamentales para la toma de decisiones informada.
En resumen, la limpieza de datos se revela como un componente esencial en este proceso.